In [1]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import torch as t
import torch.nn as nn
from torch.autograd import Variable as V
import torch.utils.data as Data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from pyfile.name_dataset import NameDataset

#### define model

In [2]:
# Parameters and DataLoaders
HIDDEN_SIZE = 100
N_CHARS = 128 
N_CLASSES = 18

In [4]:
class RNNClassifier(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, inputs):
        # Note: we run this all at once (over the whole input sequence)
        
        # input = B * S. size(0) = B
        batch_size = inputs.size(0)
        
        # input: B * S --(transpose) --> S * B
        inputs = inputs.t()
        
        # Embedding S * B --> S * B * I (embedding size)
        print("inputs size: ", inputs.size())
        embeded = self.embedding(inputs)
        print("embedding size: ", embeded.size())
        
        # Make a hidden
        hidden = self._init_hidden(batch_size)
        
        output, hidden = self.gru(embeded, hidden)
        print("gru hidden output size: ", hidden.size())
        # Use the last layer output as FC's input
        # No need th unpack, since we are going to use hidden
        fc_output = self.fc(hidden)
        print("fc output size: ", fc_output.size())
        
        return fc_output
    
    def _init_hidden(self, batch_size):
        hidden = t.zeros(self.num_layers, batch_size, self.hidden_size)
        return V(hidden)


In [16]:
# Help functions

def str2ascii_arr(msg):
    # ord: char map to ascii
    arr = [ord(c) for c in msg]
    return arr, len(arr)

# pad sequences and sort the tensor
def pad_sequences(vectorized_seqs, seq_lengths):
    seq_tensor = t.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = t.LongTensor(seq)
    return seq_tensor

# Create necessary variables, lengths, and target
def make_variables(names):
    sequence_and_length = [str2ascii_arr(name) for name in names]
    # print("sequence_and_lengths: ", sequence_and_length)
    
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = t.LongTensor([sl[1] for sl in sequence_and_length])
    # print("vectorized_seqs: ", vectorized_seqs)
    # print("seq_lengths: ", seq_lengths)
    padSequences = pad_sequences(vectorized_seqs, seq_lengths)
    return padSequences

In [17]:
# names = ['Haoyanlong', 'Xueben', 'Yanshuangying']

# pad_sequences = make_variables(names)
# print("pad_sequences: ", pad_sequences)

#### test model

In [21]:
names = ['adylov', 'solan', 'hard', 'san']
classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_CLASSES)

for name in names:
    arr, _ = str2ascii_arr(name)
    inp = V(t.LongTensor([arr]))
    output = classifier(inp)
    print("in", inp, "out", output)
    print("*************************************")

inputs = make_variables(names)
out = classifier(inputs)
print("batch in", inputs, "batch out", out)

inputs size:  torch.Size([6, 1])
embedding size:  torch.Size([6, 1, 100])
gru hidden output size:  torch.Size([1, 1, 100])
fc output size:  torch.Size([1, 1, 18])
in Variable containing:
  97  100  121  108  111  118
[torch.LongTensor of size 1x6]
 out Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.2741 -0.0228 -0.0815 -0.0937 -0.0627 -0.4570  0.3429 -0.3141 -0.0211

Columns 9 to 17 
   0.0535 -0.2229 -0.2405 -0.0655 -0.1692 -0.1103  0.0922  0.3456  0.0541
[torch.FloatTensor of size 1x1x18]

*************************************
inputs size:  torch.Size([5, 1])
embedding size:  torch.Size([5, 1, 100])
gru hidden output size:  torch.Size([1, 1, 100])
fc output size:  torch.Size([1, 1, 18])
in Variable containing:
 115  111  108   97  110
[torch.LongTensor of size 1x5]
 out Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.1535 -0.0400  0.0799  0.0614 -0.1515 -0.1010 -0.1350 -0.0872  0.1766

Columns 9 to 17 
  -0.0186  0.2866 -0.2290 -0.2586  0.1083  0.3120  0.2634  0.2101  